In [2]:
# Imports

import pandas as pd
import numpy as np
import time
import plotly.express as px
import random


In [21]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_size, state=None):
        if state is None:
            # Inital board set up
            self.state = np.random.randint(0, board_size, board_size)
        else:
            self.state = np.array(state) 

        # Set the goal fitness value where there is no queen attacks 
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
    
    
    def fitness(self):
        count = 0
        n = len(self.state)
        
        for i in range(n - 1):
            remaining_queens = self.state[i + 1:]
            current_queen = self.state[i]
            
            # Count of the queen attacks on the row 
            count += (current_queen == remaining_queens).sum()
            
            # Count of the queen attacks on the diagonal 
            distances = np.arange(1, n - i)
            upper_diagonal = current_queen + distances
            lower_diagonal = current_queen - distances
            
            # Total queen on queen attacks
            count += sum((remaining_queens == upper_diagonal) | (remaining_queens == lower_diagonal))

        # Return the final fitness 
        return self.goal_fitness - count

In [24]:
# Standard Genetic Algorithm

class Standard8QueenGA:
    def __init__(self, population_size, mutation_probability, initial_random_state):
        self.board_size = 8
        self.population_size = population_size
        self.mutation_probability = mutation_probability
        self.last_state = None

        # Initial population - unique neighbors of a random initial state
        initial_state = np.random.randint(0, self.board_size, self.board_size)
        self.current_population = [EightQueensState(self.board_size, self.random_neighbour(initial_state)) for _ in range(population_size)]


    def random_neighbour(self, state):
        new_state = state.copy() 

        # Choose a random column
        column = np.random.choice(range(self.board_size)) 

        # Create a list of possible row positions excluding the current row of the queen
        possible_rows = [row for row in range(self.board_size) if row != state[column]]

        # Set the queen in the chosen column to a new row position
        new_state[column] = np.random.choice(possible_rows)

        return new_state


    def calculate_population_fitness(self):
        # Calculate fitness and selection probability for each individual in the population
        population_data = []
        total_fitness = 0
        
        for individual in self.current_population:
            fitness = individual.fitness()
            population_data.append({
                'state': individual.state,
                'fitness': fitness
            })
            total_fitness += fitness
        
        # Calculate selection probability for each individual based on fitness
        for data in population_data:
            data['selection_prob'] = data['fitness'] / total_fitness
            
        # Return the sorted the population by the highest selection probabilities
        return sorted(population_data, key=lambda x: x['selection_prob'], reverse=True)


    def select_parent(self, population_data):
        # Generate a random number between 0 and 1
        random_value = np.random.random()
        
        # Use cumulative probability to select an individual
        cumulative_probability = 0.0
        for individual in population_data:
            cumulative_probability += individual['selection_prob']
            if cumulative_probability > random_value:
                return individual['state']


    def crossover(self, parent1, parent2):
        # Perform a single-point crossover to produce two offspring via randomisation
        idx = np.random.randint(1, len(parent1))  

        # Apply the cross over of two parents to product two children 
        return np.concatenate([parent1[:idx], parent2[idx:]]), np.concatenate([parent2[:idx], parent1[idx:]])


    def mutate(self, state):
        # Randomly mutate a queen's position in one column with a certain probability
        if np.random.random() < self.mutation_probability:

             # Select a random column to mutate
            idx = np.random.randint(len(state))  

            # Randomly choose a new row position for the queen in the selected column, excluding current row 
            state[idx] = np.random.choice(list(set(range(self.board_size)) - {state[idx]}))
        return state


    def create_next_generation(self, population_data):
        # Generate the next generation by selecting parents, applying crossover, and mutating offspring
        next_generation = []
        
        # Fill the rest of the population with offspring
        while len(next_generation) < self.population_size:
            parent1 = self.select_parent(population_data)
            parent2 = self.select_parent(population_data)
            
            # Create two children through crossover
            child1, child2 = self.crossover(parent1, parent2)
            
            # Apply mutation to each child
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            # Add children to the next generation
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        # Ensure the population size remains constant
        self.current_population = next_generation[:self.population_size]


    def run(self):
        generation = 0
        fitness_history = []
        
        while generation < 5000:
            population_data = self.calculate_population_fitness()
            
            # Check if the best individual in the population is a solution (28 for a standard chess board)
            if population_data[0]['fitness'] == 28:

                self.last_state = population_data[0]['state']
                # print(f"Solution found in generation {generation}")
                return fitness_history, generation  # Return fitness history and generation where solution is found
            
            # Track the best fitness for each generation (optional for progress analysis)
            best_fitness = population_data[0]['fitness']
            fitness_history.append(best_fitness)
            
            # Generate the next generation
            self.create_next_generation(population_data)
            
            generation += 1

        return fitness_history, generation 
    

    def display_solution(self):
        # Display the chessboard with queens in the positions defined by self.last_state
        for row in range(self.board_size):
            print(f'{self.board_size - row} |', end=' ')
            for col in range(self.board_size):
                if self.last_state[col] == row:
                    print('Q ', end='')  
                else:
                    print('- ', end='') 
            print(f'|')

In [29]:
# Run Standard Genetic Algorithm (10 trial example)

def run_8_queen_ga_solver(trails):

    for trial in range(trails):

        start_time = time.time()
        
        ga = Standard8QueenGA(
            population_size=300,
            mutation_probability=0.1,
            initial_random_state=np.random.randint(0, 8, 8)
        )
        
        ga.run()

        print(f'Solution {trial + 1}:')

        ga.display_solution()

        print(f'Run Time: {(time.time() - start_time):.2f} seconds\n')


# Run the standard 8 queen GA solver for 10 random trials
run_8_queen_ga_solver(10)

Solution 1:
8 | - - - - - Q - - |
7 | - - - Q - - - - |
6 | - Q - - - - - - |
5 | - - - - - - - Q |
4 | - - - - Q - - - |
3 | - - - - - - Q - |
2 | Q - - - - - - - |
1 | - - Q - - - - - |
Run Time: 1.58 seconds

Solution 2:
8 | - - - - Q - - - |
7 | Q - - - - - - - |
6 | - - - - - - - Q |
5 | - - - Q - - - - |
4 | - Q - - - - - - |
3 | - - - - - - Q - |
2 | - - Q - - - - - |
1 | - - - - - Q - - |
Run Time: 0.78 seconds

Solution 3:
8 | - - - Q - - - - |
7 | Q - - - - - - - |
6 | - - - - Q - - - |
5 | - - - - - - - Q |
4 | - - - - - Q - - |
3 | - - Q - - - - - |
2 | - - - - - - Q - |
1 | - Q - - - - - - |
Run Time: 1.52 seconds

Solution 4:
8 | - - - - Q - - - |
7 | - - Q - - - - - |
6 | - - - - - - - Q |
5 | - - - Q - - - - |
4 | - - - - - - Q - |
3 | Q - - - - - - - |
2 | - - - - - Q - - |
1 | - Q - - - - - - |
Run Time: 0.67 seconds

Solution 5:
8 | - - - Q - - - - |
7 | - - - - - - Q - |
6 | - - - - Q - - - |
5 | - - Q - - - - - |
4 | Q - - - - - - - |
3 | - - - - - Q - - |
2 | - - 

In [ ]:


        if initial_population == 'Random':
            # Initial population - all random individuals
            self.current_population = [EightQueensState(board_size) for _ in range(population_size)]
